## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler , LabelEncoder
from sklearn.model_selection import train_test_split , RandomizedSearchCV , GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import m2cgen
import os
import matplotlib.pyplot as plt
import joblib
from xgboost import XGBClassifier
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None)  # Show full content in each cell
pd.set_option('display.width', 1000)  # Set max width

Activity Mapper:

0 : walking
1 : jogging
2 : upstairs
3 : downstairs
4 : sitting
5 : standing
6 : sleeping

## Load Data

In [2]:
dataset_path = os.getcwd().replace('/notebook' , '') + "/dataset/"

# Read training data
imu_df1 = pd.read_csv(dataset_path + "imu_data.csv")
imu_df2 = pd.read_csv(dataset_path + "imu_data_v2.csv")
imu_df1 = imu_df1.iloc[:,1:]
imu_df2 = imu_df2.iloc[:,1:]
imu_df2 = imu_df2.drop(columns = (imu_df2.iloc[:,12:16]).columns , axis= 1)
imu_df1.columns = imu_df1.columns.str.replace("'" , "")
imu_df2.columns = imu_df2.columns.str.replace("'" , "")

imu_df = pd.concat([imu_df1, imu_df2] , axis= 0)

In [3]:
imu_df.columns = imu_df.columns.str.replace("'" , "")
x = imu_df.iloc[:,:-1]
y = imu_df.iloc[:,-1]

In [4]:
x.columns

Index(['attitude.roll', ' attitude.pitch', ' attitude.yaw', ' gravity.x', ' gravity.y', ' gravity.z', ' rotationRate.x', ' rotationRate.y', ' rotationRate.z', ' userAcceleration.x', ' userAcceleration.y', ' userAcceleration.z'], dtype='object')

# Modeling

## Motion Classification

In [5]:
"""
Map
0 - steady motion,
1 - unsteady motion
"""
mapper = {
    0 : 1,
    1 : 1,
    2 : 1,
    3 : 1,
    4 : 0,
    5 : 0,
    6 : 0
}

In [6]:
motion_y = y.map(mapper)

x1, x_test , y1 , y_test = train_test_split(x ,motion_y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [7]:
dt = DecisionTreeClassifier()

param_dist = {
    'max_depth' : [2 , 3 , 4 , 6 , 8 , 10],
    'criterion' : ['gini', 'entropy']
    }

random_search = GridSearchCV(estimator = dt,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
11,0.965757,"{'criterion': 'entropy', 'max_depth': 10}"
5,0.964569,"{'criterion': 'gini', 'max_depth': 10}"
4,0.963253,"{'criterion': 'gini', 'max_depth': 8}"
10,0.961987,"{'criterion': 'entropy', 'max_depth': 8}"
3,0.955041,"{'criterion': 'gini', 'max_depth': 6}"
9,0.952177,"{'criterion': 'entropy', 'max_depth': 6}"
2,0.933852,"{'criterion': 'gini', 'max_depth': 4}"
8,0.933414,"{'criterion': 'entropy', 'max_depth': 4}"
1,0.893813,"{'criterion': 'gini', 'max_depth': 3}"
7,0.887617,"{'criterion': 'entropy', 'max_depth': 3}"


In [8]:
dt = DecisionTreeClassifier(max_depth = 10 , criterion= 'entropy')
dt.fit(x_train , y_train)
dt.score(x_val , y_val)
dt_report = classification_report(dt.predict(x_val) , y_val)
print(dt_report)

              precision    recall  f1-score   support

           0       0.96      0.96      0.96     11030
           1       0.97      0.97      0.97     14633

    accuracy                           0.97     25663
   macro avg       0.97      0.97      0.97     25663
weighted avg       0.97      0.97      0.97     25663



In [9]:
dt_train_report = classification_report(dt.predict(x_train) , y_train)
print(dt_train_report)

              precision    recall  f1-score   support

           0       0.97      0.97      0.97     44149
           1       0.98      0.98      0.98     58500

    accuracy                           0.97    102649
   macro avg       0.97      0.97      0.97    102649
weighted avg       0.97      0.97      0.97    102649



In [10]:
dt_test_report = classification_report(dt.predict(x_test) , y_test)
print(dt_test_report)

              precision    recall  f1-score   support

           0       0.96      0.96      0.96     13771
           1       0.97      0.97      0.97     18307

    accuracy                           0.97     32078
   macro avg       0.97      0.97      0.97     32078
weighted avg       0.97      0.97      0.97     32078



In [11]:
with open(os.getcwd().replace('/notebook' , '') + "/model/motion_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(dt))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/motion_model.py" ))

64995


## Steady Movement Classification

In [12]:
steady_df  =imu_df[(imu_df['Target'] == 4) | (imu_df['Target'] == 5) | (imu_df['Target'] == 6)]
x = steady_df.iloc[:,:-1]
y = steady_df.iloc[:,-1]

x1, x_test , y1 , y_test = train_test_split(x , y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [13]:
dt = DecisionTreeClassifier()

param_dist = {
    'max_depth' : [3 , 4 , 5 , 6 , 8 , 10],
    'criterion' : ['gini', 'entropy']
    }

random_search = GridSearchCV(estimator = dt,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
5,0.896620,"{'criterion': 'gini', 'max_depth': 10}"
11,0.891467,"{'criterion': 'entropy', 'max_depth': 10}"
10,0.871217,"{'criterion': 'entropy', 'max_depth': 8}"
4,0.868675,"{'criterion': 'gini', 'max_depth': 8}"
3,0.825089,"{'criterion': 'gini', 'max_depth': 6}"
9,0.790447,"{'criterion': 'entropy', 'max_depth': 6}"
2,0.785113,"{'criterion': 'gini', 'max_depth': 5}"
8,0.759733,"{'criterion': 'entropy', 'max_depth': 5}"
7,0.703185,"{'criterion': 'entropy', 'max_depth': 4}"
1,0.694581,"{'criterion': 'gini', 'max_depth': 4}"


In [14]:
dt = DecisionTreeClassifier(max_depth = 10 , criterion= 'gini')
dt.fit(x_train , y_train)
dt.score(x_val , y_val)
dt_report = classification_report(dt.predict(x_val) , y_val)
print(dt_report)

              precision    recall  f1-score   support

           4       0.86      0.88      0.87      3581
           5       0.87      0.86      0.86      3738
           6       0.97      0.95      0.96      3694

    accuracy                           0.90     11013
   macro avg       0.90      0.90      0.90     11013
weighted avg       0.90      0.90      0.90     11013



In [15]:
dt_train_report = classification_report(dt.predict(x_train) , y_train)
print(dt_train_report)

              precision    recall  f1-score   support

           4       0.89      0.91      0.90     14395
           5       0.90      0.90      0.90     14783
           6       0.98      0.96      0.97     14873

    accuracy                           0.92     44051
   macro avg       0.92      0.92      0.92     44051
weighted avg       0.92      0.92      0.92     44051



In [16]:
dt_test_report = classification_report(dt.predict(x_test) , y_test)
print(dt_test_report)

              precision    recall  f1-score   support

           4       0.86      0.88      0.87      4487
           5       0.86      0.86      0.86      4616
           6       0.97      0.95      0.96      4663

    accuracy                           0.90     13766
   macro avg       0.90      0.90      0.90     13766
weighted avg       0.90      0.90      0.90     13766



In [17]:
with open(os.getcwd().replace('/notebook' , '') + "/model/steady_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(dt))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/steady_model.py" ))

91529


## Unsteady Movement Classification

In [18]:
unsteady_df  =imu_df[(imu_df['Target'] != 4) & (imu_df['Target'] != 5) & (imu_df['Target'] != 6)]
x = unsteady_df.iloc[:,:-1]
y = unsteady_df.iloc[:,-1]

## Unsteady Motion Classification

In [19]:
"""
Map
0 - surface motion,
1 - staircase motion
"""
mapper = {
    0 : 0,
    1 : 0,
    2 : 1,
    3 : 1
}

In [20]:
unsteady_motion_y = y.map(mapper)

x1, x_test , y1 , y_test = train_test_split(x ,unsteady_motion_y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [21]:
dt = DecisionTreeClassifier()

param_dist = {
    'max_depth' : [4 , 5 , 6 , 7 , 8 , 10],
    'criterion' : ['gini', 'entropy']
    }

random_search = GridSearchCV(estimator = dt,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
5,0.848288,"{'criterion': 'gini', 'max_depth': 10}"
11,0.844244,"{'criterion': 'entropy', 'max_depth': 10}"
4,0.836360,"{'criterion': 'gini', 'max_depth': 8}"
10,0.829141,"{'criterion': 'entropy', 'max_depth': 8}"
3,0.821291,"{'criterion': 'gini', 'max_depth': 7}"
9,0.816052,"{'criterion': 'entropy', 'max_depth': 7}"
2,0.788627,"{'criterion': 'gini', 'max_depth': 6}"
8,0.784924,"{'criterion': 'entropy', 'max_depth': 6}"
1,0.749309,"{'criterion': 'gini', 'max_depth': 5}"
7,0.745503,"{'criterion': 'entropy', 'max_depth': 5}"


In [22]:
dt = DecisionTreeClassifier(max_depth = 10 , criterion= 'gini')
dt.fit(x_train , y_train)
dt.score(x_val , y_val)
dt_report = classification_report(dt.predict(x_val) , y_val)
print(dt_report)

              precision    recall  f1-score   support

           0       0.85      0.87      0.86      7883
           1       0.84      0.83      0.84      6767

    accuracy                           0.85     14650
   macro avg       0.85      0.85      0.85     14650
weighted avg       0.85      0.85      0.85     14650



In [23]:
dt_train_report = classification_report(dt.predict(x_train) , y_train)
print(dt_train_report)

              precision    recall  f1-score   support

           0       0.88      0.89      0.89     31818
           1       0.87      0.86      0.86     26780

    accuracy                           0.88     58598
   macro avg       0.88      0.87      0.88     58598
weighted avg       0.88      0.88      0.88     58598



In [24]:
dt_test_report = classification_report(dt.predict(x_test) , y_test)
print(dt_test_report)

              precision    recall  f1-score   support

           0       0.86      0.87      0.86      9951
           1       0.84      0.83      0.84      8361

    accuracy                           0.85     18312
   macro avg       0.85      0.85      0.85     18312
weighted avg       0.85      0.85      0.85     18312



In [25]:
with open(os.getcwd().replace('/notebook' , '') + "/model/unsteady_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(dt))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/unsteady_model.py" ))

98054


## Staircase motion

In [26]:
staircase_df  =imu_df[(imu_df['Target'] == 1) | (imu_df['Target'] == 2)]
x = staircase_df.iloc[:,:-1]
y = staircase_df.iloc[:,-1]

x1, x_test , y1 , y_test = train_test_split(x , y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [27]:
dt = DecisionTreeClassifier()

param_dist = {
    'max_depth' : [4 , 5 , 6 , 7 , 8 , 10],
    'criterion' : ['gini', 'entropy']
    }

random_search = GridSearchCV(estimator = dt,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
5,0.920234,"{'criterion': 'gini', 'max_depth': 10}"
11,0.918753,"{'criterion': 'entropy', 'max_depth': 10}"
4,0.907962,"{'criterion': 'gini', 'max_depth': 8}"
10,0.904965,"{'criterion': 'entropy', 'max_depth': 8}"
3,0.896467,"{'criterion': 'gini', 'max_depth': 7}"
9,0.896114,"{'criterion': 'entropy', 'max_depth': 7}"
8,0.874603,"{'criterion': 'entropy', 'max_depth': 6}"
2,0.870478,"{'criterion': 'gini', 'max_depth': 6}"
1,0.851964,"{'criterion': 'gini', 'max_depth': 5}"
7,0.847591,"{'criterion': 'entropy', 'max_depth': 5}"


In [28]:
dt = DecisionTreeClassifier(max_depth = 10 , criterion= 'gini')
dt.fit(x_train , y_train)
dt.score(x_val , y_val)
dt_report = classification_report(dt.predict(x_val) , y_val)
print(dt_report)

              precision    recall  f1-score   support

           1       0.92      0.93      0.92      3732
           2       0.92      0.91      0.92      3358

    accuracy                           0.92      7090
   macro avg       0.92      0.92      0.92      7090
weighted avg       0.92      0.92      0.92      7090



In [29]:
dt_train_report = classification_report(dt.predict(x_train) , y_train)
print(dt_train_report)

              precision    recall  f1-score   support

           1       0.95      0.96      0.95     14905
           2       0.95      0.95      0.95     13453

    accuracy                           0.95     28358
   macro avg       0.95      0.95      0.95     28358
weighted avg       0.95      0.95      0.95     28358



In [30]:
dt_test_report = classification_report(dt.predict(x_test) , y_test)
print(dt_test_report)

              precision    recall  f1-score   support

           1       0.91      0.93      0.92      4603
           2       0.92      0.91      0.91      4259

    accuracy                           0.92      8862
   macro avg       0.92      0.92      0.92      8862
weighted avg       0.92      0.92      0.92      8862



In [31]:
with open(os.getcwd().replace('/notebook' , '') + "/model/staircase_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(dt))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/staircase_model.py" ))

74353


## Surface motion classification

In [32]:
surface_df  =imu_df[(imu_df['Target'] == 0) | (imu_df['Target'] == 4)]
x = surface_df.iloc[:,:-1]
y = surface_df.iloc[:,-1]

x1, x_test , y1 , y_test = train_test_split(x , y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [33]:
dt = DecisionTreeClassifier()

param_dist = {
    'max_depth' : [4 , 5 , 6 , 7 , 8 , 10],
    'criterion' : ['gini', 'entropy']
    }

random_search = GridSearchCV(estimator = dt,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

,mean_test_score,params
5,0.971768,"{'criterion': 'gini', 'max_depth': 10}"
4,0.970699,"{'criterion': 'gini', 'max_depth': 8}"
3,0.970353,"{'criterion': 'gini', 'max_depth': 7}"
11,0.970008,"{'criterion': 'entropy', 'max_depth': 10}"
10,0.969379,"{'criterion': 'entropy', 'max_depth': 8}"
2,0.967996,"{'criterion': 'gini', 'max_depth': 6}"
9,0.966455,"{'criterion': 'entropy', 'max_depth': 7}"
8,0.964914,"{'criterion': 'entropy', 'max_depth': 6}"
1,0.964694,"{'criterion': 'gini', 'max_depth': 5}"
0,0.960419,"{'criterion': 'gini', 'max_depth': 4}"


In [34]:
dt = DecisionTreeClassifier(max_depth = 10 , criterion= 'gini')
dt.fit(x_train , y_train)
dt.score(x_val , y_val)
dt_report = classification_report(dt.predict(x_val) , y_val)
print(dt_report)

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      4272
           4       0.97      0.97      0.97      3680

    accuracy                           0.97      7952
   macro avg       0.97      0.97      0.97      7952
weighted avg       0.97      0.97      0.97      7952



In [35]:
dt_train_report = classification_report(dt.predict(x_train) , y_train)
print(dt_train_report)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     17069
           4       0.99      0.98      0.98     14739

    accuracy                           0.99     31808
   macro avg       0.99      0.99      0.99     31808
weighted avg       0.99      0.99      0.99     31808



In [36]:
dt_test_report = classification_report(dt.predict(x_test) , y_test)
print(dt_test_report)

              precision    recall  f1-score   support

           0       0.97      0.98      0.98      5308
           4       0.98      0.97      0.97      4632

    accuracy                           0.98      9940
   macro avg       0.98      0.97      0.98      9940
weighted avg       0.98      0.98      0.98      9940



In [37]:
with open(os.getcwd().replace('/notebook' , '') + "/model/surface_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(dt))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/surface_model.py" ))

42125
